In [7]:
import requests
import zipfile
import re
import random

In [2]:
r = requests.get("https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("sms.zip", "wb") as f:
  f.write(r.content)
zipfile.ZipFile("sms.zip").extractall("./")

In [ ]:
# load data

data = []
with open("SMSSpamCollection.txt", "r") as f:
  for line in f:
    cls, txt = line.strip().split("\t")
    bow = set(re.findall("[0-9a-z_]+", txt.lower()))
    data.append([cls, bow])
  
data

In [8]:
# split data into train and test

random.shuffle(data)
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train = data[:train_size]
test = data[train_size:]

In [22]:
# train 데이터에서, naive bayes 계산을 위한 값들을 미리 다 계산해놓기
# prior prob. P(spam), P(ham)
# P(spam) = spam 문자 수 / 전체 문자 수
# 라플라스 스무딩 -> 관측한 메일에서 한번도 등장하지 않은 단어가 있으면, 예측할 때 결과가 좀 이상해즌 눔ㄴ제

n_total = train_size
n_spam = sum(1 for cls, bow in train if cls == 'spam')
n_ham = sum(1 for cls, bow in train if cls == 'ham')

alpha = 0.1

prior_spam = (n_spam + alpha) / (n_total + 2*alpha)
prior_ham = (n_ham + alpha) / (n_total + 2*alpha)

print(prior_spam, prior_ham)

0.1347999641191245 0.8652000358808755


In [23]:
# 모든 단어의 likelihood, P(w|spam), P(w|ham)
# P(w|spam) = w를 포함하는 spam 문자 수 + alpha / (전체 spam 문자 수 + alpha)

'''
spam_words = {}
ham_words = {}

for cls, bow in train:
  for word in bow:
    if cls == 'spam':
      if word not in spam_words:
        spam_words[word] = 0
      spam_words[word] += 1
    else:
      if word not in ham_words:
        ham_words[word] = 0
      ham_words[word] += 1
'''
from collections import Counter

spam_words = Counter(word for cls, bow in train
                     for word in bow
                     if cls == 'spam')
ham_words = Counter(word for cls, bow in train
                    for word in bow
                    if cls == 'ham')

spam_words

Counter({'txt': 120,
         'moby': 4,
         'unsub': 7,
         'arrow': 1,
         'who': 23,
         'a': 237,
         'u': 110,
         '1': 90,
         'know': 15,
         'win': 53,
         'will': 36,
         'be': 40,
         'of': 67,
         'stop': 87,
         'sp': 8,
         'to': 377,
         '82277': 7,
         '100': 32,
         'name': 9,
         'the': 132,
         'duchess': 2,
         'high': 2,
         'quiz': 15,
         'pub': 2,
         'her': 6,
         'new': 56,
         '50': 48,
         'first': 6,
         'street': 2,
         'if': 28,
         'cornwall': 2,
         '008704050406': 2,
         'prize': 70,
         'call': 271,
         'code': 24,
         'statement': 12,
         'private': 13,
         '2004': 6,
         '7': 8,
         'unredeemed': 3,
         '05': 2,
         '45239': 3,
         'bonus': 16,
         '078498': 1,
         'shows': 27,
         '06': 7,
         'claim': 93,
         '786': 3,
   

In [24]:
def predict(bow):
  # spam_score = ?
  # ham_score = ?
  spam_score = prior_spam
  ham_score = prior_ham
  for word in bow:
    spam_score += (spam_words[word] + alpha)/(n_spam + alpha)
    ham_score += (ham_words[word] + alpha)/(n_ham + alpha)

  if spam_score < ham_score:
    return "ham"
  else:
    return "spam"


In [25]:
tp, tn, fp, fn = 0, 0, 0, 0

for ans, bow in test:
  pred = predict(bow)
  if pred == 'spam':
    if ans == 'spam':
      tp += 1
    else:
      fp += 1
  else:
    if ans == 'ham':
      tn += 1
    else:
      fn += 1

print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")

tp: 126, tn: 950, fp: 19, fn: 20


In [21]:
# 맞힌 수 / 전체 테스트 수
acc = (tp + tn) / (tp + tn + fp + fn)
# 스팸 판정 중 실제 스팸 수 / 스팸 판정
prec = tp / (tp + fp)
# 스팸 판정 수 / 실제 스팸 수
recall = tp / (tp + fn)
f1 = 2 * prec * recall / (prec + recall)
print(f"acc: {acc:.3f}, prec: {prec:.3f}, reacall: {recall:.3f}, f1: {f1:.3f}")

acc: 0.966, prec: 0.8698630136986302, reacall: 0.8698630136986302, f1: 0.8698630136986303
